In [1]:
import numpy as np 
import pandas as pd 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

import warnings
warnings.filterwarnings('ignore')

In [4]:
# checkin = pd.read_json('yelp_academic_dataset_business.json', lines=True)
business = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
reviews = pd.read_csv('yelp_dataset/Yelp_Reviews_Subset.csv')

In [5]:
reviews = reviews.drop(columns=['useful','funny','cool'])
reviews.shape

(1268552, 6)

In [241]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [242]:
state_isopen = business[['business_id', 'state', 'is_open', 'categories', 'latitude', 'longitude', 'stars', 'review_count']]
# cond = state_isopen['is_open'] == 1 and ((state_isopen['state'] == 'PA') or (state_isopen['state'] == 'FL') )
opened = state_isopen[state_isopen['is_open'] == 1]
pa_fl = opened[(opened['state'] == 'PA') | (opened['state'] == 'FL')]
pa_fl

,business_id,state,is_open,categories,latitude,longitude,stars,review_count
3,MTSW4McQd7CbVtyjqoe9mw,PA,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",39.955505,-75.155564,4.0,80
4,mWMc6_wTdE0EUBKIGXDVfA,PA,1,"Brewpubs, Breweries, Food",40.338183,-75.471659,4.5,13
7,qkRM_2X51Yqxk3btlwAQIg,FL,1,"Synagogues, Religious Organizations",27.766590,-82.732983,3.5,5
10,UJsufbvfyfONHeWdvAHKjA,FL,1,"Department Stores, Shopping, Fashion",28.190459,-82.457380,3.5,6
11,eEOYSgkmpB90uNA7lDOMRA,FL,1,"Vietnamese, Food, Restaurants, Food Trucks",27.955269,-82.456320,4.0,10
...,...,...,...,...,...,...,...,...
150326,JDM7kiCEmeBc_5W-z6eZZw,PA,1,"Real Estate, Apartments, Home Services",39.968806,-75.180608,3.0,6
150329,9U1Igcpe954LoWZRmNc-zg,PA,1,"Day Spas, Beauty & Spas, Skin Care, Massage",39.932756,-75.144504,3.0,32
150334,LJ4GjQ1HL6kqvIPpNUNNaQ,PA,1,"Health & Medical, Yoga, Shopping, Naturopathic...",39.945966,-75.169666,4.5,39
150336,WnT9NIzQgLlILjPT0kEcsQ,PA,1,"Restaurants, Mexican",39.935982,-75.158665,4.5,35


In [243]:
# reviews.merge(pa_fl, on = 'business_id')

In [244]:
pa_fl.head()

,business_id,state,is_open,categories,latitude,longitude,stars,review_count
3,MTSW4McQd7CbVtyjqoe9mw,PA,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",39.955505,-75.155564,4.0,80
4,mWMc6_wTdE0EUBKIGXDVfA,PA,1,"Brewpubs, Breweries, Food",40.338183,-75.471659,4.5,13
7,qkRM_2X51Yqxk3btlwAQIg,FL,1,"Synagogues, Religious Organizations",27.766590,-82.732983,3.5,5
10,UJsufbvfyfONHeWdvAHKjA,FL,1,"Department Stores, Shopping, Fashion",28.190459,-82.457380,3.5,6
11,eEOYSgkmpB90uNA7lDOMRA,FL,1,"Vietnamese, Food, Restaurants, Food Trucks",27.955269,-82.456320,4.0,10


In [245]:
reviews.head().T

,0,1,2,3,4
review_id,VpibERhdmOvSagzKSdVuSw,j6U6UkRwhXAH75HaCBlItg,jNqsthPdQZGi1Ty6KI7C0A,3Np9IPeJi83GFNbT2bZ9EQ,PShXXvJDxYenqvvUUj9pDQ
user_id,sWVVqJQAl6w6cOzbUsUC3A,zge9NVRIAZCUP2-kLL7wEw,ukqwvn2e_RffCu-EElhGvw,XvO5fcjWNHAIgIbUo3xIbg,NbT6S9Z4fqQRX7HDHYnD1g
business_id,tsx84z4c0B-y6J5fqfvBqg,tsx84z4c0B-y6J5fqfvBqg,tsx84z4c0B-y6J5fqfvBqg,tsx84z4c0B-y6J5fqfvBqg,tsx84z4c0B-y6J5fqfvBqg
stars,5.0,4.0,5.0,5.0,5.0
text,Ty was amazing it was the first time and he ga...,"Very good, we've been here 3 times now so far ...",I love this location. No matter how busy they ...,"I had the shrimp and my husband had the crab, ...",Our second visit to Mr. & Mrs. Crab. Food is d...
date,2020-01-15 12:59:01,2020-01-14 16:07:20,2020-02-09 19:57:12,2019-12-08 02:42:09,2020-01-09 00:26:47
state,FL,FL,FL,FL,FL
is_open,1,1,1,1,1
categories,"Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...","Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...","Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...","Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...","Nightlife, Cajun/Creole, Bars, Cocktail Bars, ..."
longitude,-82.68541,-82.68541,-82.68541,-82.68541,-82.68541


In [246]:
rest_business = pa_fl[pa_fl.categories.fillna('-').str.lower().str.contains('restaurant')]
rest_business.shape

(13993, 8)

In [247]:
rest_business

,business_id,state,is_open,categories,latitude,longitude,stars,review_count
3,MTSW4McQd7CbVtyjqoe9mw,PA,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",39.955505,-75.155564,4.0,80
11,eEOYSgkmpB90uNA7lDOMRA,FL,1,"Vietnamese, Food, Restaurants, Food Trucks",27.955269,-82.456320,4.0,10
15,MUTTqe8uqyMdBl186RmNeA,PA,1,"Sushi Bars, Restaurants, Japanese",39.953949,-75.143226,4.0,245
19,ROeacJQwBeh05Rqg7F6TCg,PA,1,"Korean, Restaurants",39.943223,-75.162568,4.5,205
33,kV_Q1oqis8Qli8dUoGpTyQ,PA,1,"Pizza, Restaurants",40.006707,-75.289671,3.5,109
...,...,...,...,...,...,...,...,...
150292,esBGrrmuZzSiECyRBoKvvA,FL,1,"Bars, Beer Bar, Nightlife, Wine Bars, Pizza, R...",27.770872,-82.643069,4.5,38
150297,Wko0HNelt55ZIp3Dm3eD2g,PA,1,"Convenience Stores, Sandwiches, Restaurants, F...",39.920225,-75.325974,3.5,5
150306,wVxXRFf10zTTAs11nr4xeA,PA,1,"Restaurants, Specialty Food, Food, Sandwiches,...",40.032483,-75.214430,3.0,55
150325,l9eLGG9ZKpLJzboZq-9LRQ,PA,1,"Restaurants, Sandwiches, Convenience Stores, C...",39.925656,-75.310344,3.0,11


## Add in the KMeans Location Based Features

In [248]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [249]:
np.sqrt(13993/2)

83.64508353752778

In [250]:
# Rule of thumb for k: sqrt(n/2); here n is 112390 - total no. of NYC events
kmeans = KMeans(n_clusters=83, init='k-means++')

# Compute the clusters based on longitude and latitude features
X_sample = rest_business[['latitude','longitude']].sample(frac=0.1)
kmeans.fit(X_sample)
y = kmeans.labels_
print("k = 83", " silhouette_score ", silhouette_score(X_sample, y, metric='euclidean'))

k = 83  silhouette_score  0.4424617970412674


In [251]:
rest_business['cluster'] = kmeans.predict(rest_business[['latitude','longitude']])

In [252]:
rest_business

,business_id,state,is_open,categories,latitude,longitude,stars,review_count,cluster
3,MTSW4McQd7CbVtyjqoe9mw,PA,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",39.955505,-75.155564,4.0,80,48
11,eEOYSgkmpB90uNA7lDOMRA,FL,1,"Vietnamese, Food, Restaurants, Food Trucks",27.955269,-82.456320,4.0,10,81
15,MUTTqe8uqyMdBl186RmNeA,PA,1,"Sushi Bars, Restaurants, Japanese",39.953949,-75.143226,4.0,245,45
19,ROeacJQwBeh05Rqg7F6TCg,PA,1,"Korean, Restaurants",39.943223,-75.162568,4.5,205,48
33,kV_Q1oqis8Qli8dUoGpTyQ,PA,1,"Pizza, Restaurants",40.006707,-75.289671,3.5,109,9
...,...,...,...,...,...,...,...,...,...
150292,esBGrrmuZzSiECyRBoKvvA,FL,1,"Bars, Beer Bar, Nightlife, Wine Bars, Pizza, R...",27.770872,-82.643069,4.5,38,11
150297,Wko0HNelt55ZIp3Dm3eD2g,PA,1,"Convenience Stores, Sandwiches, Restaurants, F...",39.920225,-75.325974,3.5,5,36
150306,wVxXRFf10zTTAs11nr4xeA,PA,1,"Restaurants, Specialty Food, Food, Sandwiches,...",40.032483,-75.214430,3.0,55,49
150325,l9eLGG9ZKpLJzboZq-9LRQ,PA,1,"Restaurants, Sandwiches, Convenience Stores, C...",39.925656,-75.310344,3.0,11,36


In [173]:
# reviews for filtered business: only restaurant, only PA and FL, only opened
reviews = reviews.merge(rest_business, on = 'business_id')

In [156]:
from sklearn.feature_extraction.text import CountVectorizer

categories = rest_business['categories'].str.split(',')

categories_words = []

for w in categories:
    categories_words += w

all_words = list(pd.Series(categories_words).str.strip(' '))

vectorizer = CountVectorizer(max_df=0.5)

# Fit and transform the data to get the bag of words representation
bow = vectorizer.fit_transform(all_words)

# Print the vocabulary and the bag of words representation
print('Vocabulary:', vectorizer.vocabulary_)
print('Bag of words:\n', bow.toarray())

Vocabulary: {'restaurants': 484, 'food': 238, 'bubble': 73, 'tea': 575, 'coffee': 129, 'bakeries': 35, 'vietnamese': 629, 'trucks': 608, 'sushi': 563, 'bars': 41, 'japanese': 327, 'korean': 340, 'pizza': 436, 'eatertainment': 201, 'arts': 26, 'entertainment': 209, 'brewpubs': 69, 'american': 15, 'traditional': 599, 'breweries': 68, 'italian': 325, 'sports': 546, 'nightlife': 400, 'steakhouses': 550, 'salad': 493, 'beer': 48, 'bar': 38, 'burgers': 78, 'lounges': 365, 'wraps': 650, 'new': 398, 'automotive': 32, 'delis': 174, 'gas': 252, 'stations': 549, 'sandwiches': 497, 'convenience': 141, 'stores': 553, 'french': 240, 'moroccan': 391, 'seafood': 502, 'mediterranean': 380, 'chinese': 115, 'mexican': 384, 'fast': 223, 'chicken': 113, 'shop': 515, 'thai': 584, 'caterers': 103, 'bagels': 34, 'event': 215, 'planning': 437, 'services': 507, 'pubs': 465, 'irish': 322, 'music': 394, 'venues': 625, 'breakfast': 67, 'brunch': 72, 'vegan': 620, 'pub': 463, 'roasteries': 489, 'cupcakes': 163, 'ca

### End of pre-processing

## Restaurant Recommendation System Using Latent Factor Collaborative Filtering

In [174]:
reviews.head()

,review_id,user_id,business_id,stars,text,date,state,is_open,categories,longitude,latitude,cluster
0,VpibERhdmOvSagzKSdVuSw,sWVVqJQAl6w6cOzbUsUC3A,tsx84z4c0B-y6J5fqfvBqg,5.0,Ty was amazing it was the first time and he ga...,2020-01-15 12:59:01,FL,1,"Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...",-82.68541,27.840642,24
1,j6U6UkRwhXAH75HaCBlItg,zge9NVRIAZCUP2-kLL7wEw,tsx84z4c0B-y6J5fqfvBqg,4.0,"Very good, we've been here 3 times now so far ...",2020-01-14 16:07:20,FL,1,"Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...",-82.68541,27.840642,24
2,jNqsthPdQZGi1Ty6KI7C0A,ukqwvn2e_RffCu-EElhGvw,tsx84z4c0B-y6J5fqfvBqg,5.0,I love this location. No matter how busy they ...,2020-02-09 19:57:12,FL,1,"Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...",-82.68541,27.840642,24
3,3Np9IPeJi83GFNbT2bZ9EQ,XvO5fcjWNHAIgIbUo3xIbg,tsx84z4c0B-y6J5fqfvBqg,5.0,"I had the shrimp and my husband had the crab, ...",2019-12-08 02:42:09,FL,1,"Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...",-82.68541,27.840642,24
4,PShXXvJDxYenqvvUUj9pDQ,NbT6S9Z4fqQRX7HDHYnD1g,tsx84z4c0B-y6J5fqfvBqg,5.0,Our second visit to Mr. & Mrs. Crab. Food is d...,2020-01-09 00:26:47,FL,1,"Nightlife, Cajun/Creole, Bars, Cocktail Bars, ...",-82.68541,27.840642,24


In [176]:
#Select only stars and text

# import nltk
# nltk.download('stopwords')

yelp_data = reviews[['business_id', 'user_id', 'stars', 'text']]
yelp_data = yelp_data.sample(50000, random_state=42)

import string
from nltk.corpus import stopwords
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [177]:
yelp_data

,business_id,user_id,stars,text
275088,CnNcsICrYvSMUcl7dZoCCg,pGtGZpe0iPjndIJzd84oNg,5.0,I saw an ad for The Spot on Facebook and saw t...
155750,w01MzdFQgP7j7Rn8em_lkA,yWoozHIUHNlRg2VY3OcfKQ,2.0,We went last night and although the food was g...
87562,ChMvzo7WYSZzxlwGMS7CFA,XELljeWWO5NTnIWcXbajjg,4.0,Very nice Mexican food. It came recommended by...
127728,l8gAoQqqVfphPe9jmIJZ3g,EHlMuFlW2Jq6fxuzdeb5EA,5.0,Our waiter was awesome because we were a group...
160559,U_U6C9AF7xmQijTTUFRzkQ,CoSmlV9kohWoh6G3IC7D5g,2.0,"The staff was extremely friendly, but other th..."
...,...,...,...,...
263572,QBnteiO92wOvE1WCYjMC_w,ZssWomaPLdZWeOumPNmcsQ,5.0,"Yummy food! Nice, friendly employees. Their bu..."
56966,V6wr6ibKSuPm9x5PMZOufA,5Nl6vX1LSbd3K1KBkOVtxQ,4.0,I would give it it 3 1/2 stars as far as food ...
74139,XRZaQ9-jEJobDnFzFbHheQ,3t9g8eT90JSjcCXMcF8adw,4.0,Food was really good! I had the Chicken Shawar...
283423,cFSyJluKa2SHtgMMvlx6SQ,5JeJ0cI-ewDBRjyqu8k9dw,3.0,"For a $20 dollar pizza, you better show up for..."


In [178]:
def text_process(mess):
    remove_punc = [char for char in mess if char not in string.punctuation]
    remove_punc = ''.join(remove_punc)
    return " ".join([word for word in remove_punc.split() if word.lower() not in stop])

yelp_data['text'] = yelp_data['text'].apply(text_process)

yelp_data

,business_id,user_id,stars,text
275088,CnNcsICrYvSMUcl7dZoCCg,pGtGZpe0iPjndIJzd84oNg,5.0,saw ad Spot Facebook saw offering pickle sandw...
155750,w01MzdFQgP7j7Rn8em_lkA,yWoozHIUHNlRg2VY3OcfKQ,2.0,went last night although food good service exc...
87562,ChMvzo7WYSZzxlwGMS7CFA,XELljeWWO5NTnIWcXbajjg,4.0,nice Mexican food came recommended local final...
127728,l8gAoQqqVfphPe9jmIJZ3g,EHlMuFlW2Jq6fxuzdeb5EA,5.0,waiter awesome group 12 nieces graduation atte...
160559,U_U6C9AF7xmQijTTUFRzkQ,CoSmlV9kohWoh6G3IC7D5g,2.0,staff extremely friendly breakfast average got...
...,...,...,...,...
263572,QBnteiO92wOvE1WCYjMC_w,ZssWomaPLdZWeOumPNmcsQ,5.0,Yummy food Nice friendly employees burritos be...
56966,V6wr6ibKSuPm9x5PMZOufA,5Nl6vX1LSbd3K1KBkOVtxQ,4.0,would give 3 12 stars far food goes able Tampa...
74139,XRZaQ9-jEJobDnFzFbHheQ,3t9g8eT90JSjcCXMcF8adw,4.0,Food really good Chicken Shawarma FalafelI thi...
283423,cFSyJluKa2SHtgMMvlx6SQ,5JeJ0cI-ewDBRjyqu8k9dw,3.0,20 dollar pizza better show taste texture husb...


In [179]:
# Separate into user and business with review
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [180]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

business_df.head()

,text
business_id,
---kPU91CF4Lq2-WlRu9Lw,food truck alongside picnic area atmosphere fi...
--epgcb7xHGuJ-4PUeSLAw,Called order take brunch mom Unfortunately man...
-0EdehHjIQc0DtYU8QcAig,Would leave zero starts could used love place ...
-0FX23yAacC4bbLaGPvyxw,went last night first time absolutely amazing ...
-0TffRSXXIlBYVbb5AwfTg,Came postpresentation celebration coworkers re...


In [181]:
import nltk

word_data = "It originated from the idea that there are readers who prefer learning new skills from the comforts of their drawing rooms"
nltk_tokens = nltk.word_tokenize(word_data)
print (nltk_tokens)

['It', 'originated', 'from', 'the', 'idea', 'that', 'there', 'are', 'readers', 'who', 'prefer', 'learning', 'new', 'skills', 'from', 'the', 'comforts', 'of', 'their', 'drawing', 'rooms']


In [182]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer

#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])
# userid_vectors = pd.DataFrame.sparse.from_spmatrix(userid_vectors, index=userid_df.index)
userid_vectors = pd.DataFrame(userid_vectors.todense(), index=userid_df.index)

#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])
# businessid_vectors = pd.DataFrame.sparse.from_spmatrix(businessid_vectors, index=business_df.index)
businessid_vectors = pd.DataFrame(businessid_vectors.todense(), index=business_df.index)

In [183]:
businessid_vectors

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
business_id,,,,,,,,,,,,,,,,,,,,,
---kPU91CF4Lq2-WlRu9Lw,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
--epgcb7xHGuJ-4PUeSLAw,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
-0EdehHjIQc0DtYU8QcAig,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.067137,0.0,0.0,0.0,0.0
-0FX23yAacC4bbLaGPvyxw,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
-0TffRSXXIlBYVbb5AwfTg,0.0,0.013568,0.000000,0.01735,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.014265,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zxRmQ_FWVowh8rlzLCSURQ,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
zxqCNze-DDqEZIYrEMJLxg,0.0,0.000000,0.123084,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
zxtkbSchfaqY1TpZ7KnmKA,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [184]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])
userid_rating_matrix.head()

business_id,---kPU91CF4Lq2-WlRu9Lw,--epgcb7xHGuJ-4PUeSLAw,-0EdehHjIQc0DtYU8QcAig,-0FX23yAacC4bbLaGPvyxw,-0TffRSXXIlBYVbb5AwfTg,-1B9pP_CrRBJYPICE5WbRA,-1oygVebK81K8JEPI6H6Lw,-2YZIUArmrIMVVBi8CT47w,-2dvQxx3cYXd5XmFdDDsDA,-2wh7NTLkWEgsrLJvilnFQ,...,zvvl3c1FO3O3BZdhusficA,zwBvdDWFQpYJE1UMDTlSyw,zwJWZxRRNkEcQ3zfVyqMFQ,zwXT4m9svdg-xh2cKjEp-Q,zwd4dyQ5ovnjVojWfAuhMw,zxRmQ_FWVowh8rlzLCSURQ,zxqCNze-DDqEZIYrEMJLxg,zxtkbSchfaqY1TpZ7KnmKA,zzIF9qp2UoHN48EeZH_IDg,zzjFdJwXuxBOGe9JeY_EMw
user_id,,,,,,,,,,,,,,,,,,,,,
--WLSCJPoCNQmTmG2yPAcQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--bK3rF1_DLb1jRbdCbHzQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--kgi4AWzsva7ekDTdEhWA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--u09WAjW741FdfkJXxNmg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-00OOWci26LPDUO3fNdtXQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
from scipy.sparse import csr_matrix
sparse_urm = csr_matrix(userid_rating_matrix.fillna(0))

In [186]:
sparse_urm

<36503x10074 sparse matrix of type '<class 'numpy.float64'>'
	with 49824 stored elements in Compressed Sparse Row format>

In [187]:
def sparseMF(R, P, Q, steps=10, gamma=0.001, lamda=0.02, batch_size = 100, decay_rate=0.0):
    idx = R.indptr
    prev = 0
    new_prev = 0
    for step in range(steps): 
        for pointer in range(1, len(idx)):
            num_elem = idx[pointer]
            for _ in range(prev, num_elem):
                i = userid_rating_matrix.index[pointer-1]
                col = R.indices[_]
                j = userid_rating_matrix.columns[col]
                R_val = R.data[_]
#                 print([i, j])
                eij = R_val - np.dot(P.loc[i], Q.loc[j])
                P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
            prev = num_elem
        
        e=0
        for ptr in range(1, len(idx)):
            num_ = idx[ptr]
            for _ in range(new_prev, num_):
                # if R.iloc[i,j]>0:
                i2 = userid_rating_matrix.index[ptr-1]
                col = R.indices[_]
                j2 = userid_rating_matrix.columns[col]
                e = e + pow(R.data[_]-np.dot(P.loc[i2],Q.loc[j2]),2) + lamda*(pow(np.linalg.norm(P.loc[i2]),2) +pow(np.linalg.norm(Q.loc[j2]),2))
            new_prev = num_
        if e<0.001:
            break
    return P, Q

In [188]:
%%time
sparse_P, sparse_Q = sparseMF(sparse_urm, userid_vectors, businessid_vectors, steps=25, gamma=0.001,lamda=0.02)


CPU times: user 47min 23s, sys: 1min 44s, total: 49min 8s
Wall time: 16min 31s


In [189]:
sparse_P.to_csv('sparse_P.csv')

In [190]:
sparse_Q.to_csv('sparse_Q.csv')

In [191]:
# print(csr_matrix(sparse_Q))

In [271]:
def loc_recommend(df, location):
    latitude = location[0]
    longitude = location[1]
    predicted_cluster = kmeans.predict(np.array([latitude, longitude]).reshape(1,-1))[0]
    # Fetch the venue name of the top most record in the topvenues dataframe for the predicted cluster
    restaurants = df[df['cluster']==predicted_cluster]
#     .sort_values(by = ['stars', 'review_count'], ascending = False)
    return restaurants

In [272]:
# rest_business

In [273]:
# retrieve the recommand restaurants base on location


In [274]:
user_input = "sushi"
location = [40, -75]

loc_base = loc_recommend(rest_business, location)
loc_base.head()

request_df = pd.DataFrame().assign(text = [user_input])

request_df['text'] = request_df['text'].apply(text_process)

request_vector = userid_vectorizer.transform(request_df['text'])

request_v_df = pd.DataFrame(request_vector.toarray(), index=request_df.index, columns=userid_vectorizer.get_feature_names_out())
spars_rec = pd.DataFrame(np.dot(request_v_df.loc[0],sparse_Q.T),index=sparse_Q.index,columns=['Rating']).sort_values(by='Rating', ascending=False)

spars_rec = spars_rec[spars_rec['Rating'] > 0]
spars_rec

,Rating
business_id,
TunmRrfZb7bt53T6HJi4UQ,1.610722e+00
zGtHhmjmZV-ZTTVw5QlTKg,1.261080e+00
AauDx7QSes9nZOAAa4LIbQ,9.867232e-01
61fGIOE0DxjkZXE5wFDTqg,9.360826e-01
ZGChyjzwb_buWTT6Wg-q7g,8.875186e-01
...,...
BtCjmQF1TFIooIv8oksAQA,2.027030e-18
Lqm-iHcnvu3GXbm2kHhfNw,1.062416e-18
BaB6bGXtiZCSbJ1g5ZLmKA,8.234269e-19


In [275]:
combined_top5 = spars_rec.merge(loc_base, on='business_id').sort_values(by = ['Rating', 'stars', 'review_count'], ascending = False)[:5]
combined_top5



,business_id,Rating,state,is_open,categories,latitude,longitude,stars,review_count,cluster
0,ZGChyjzwb_buWTT6Wg-q7g,0.887519,PA,1,"Japanese, Sushi Bars, Restaurants",40.102460,-75.028764,4.0,122,71
1,YAUUdbESr8BYQ_TFgDjT5Q,0.462362,PA,1,"Asian Fusion, Sushi Bars, Japanese, Steakhouse...",40.086599,-75.037949,4.0,209,71
2,AEAW8IkEQv8wV6lXKfx06w,0.379267,PA,1,"Seafood, Asian Fusion, Chicken Wings, Restaura...",40.083960,-75.035514,4.5,43,71
3,D48I-imYNjFo9bmsiZc-YQ,0.311456,PA,1,"Steakhouses, Sushi Bars, Japanese, Restaurants...",40.064934,-74.983616,4.0,97,71
4,shbtvcGnl-PEQOpffwYq-w,0.287405,PA,1,"Restaurants, Japanese, Chinese",40.069086,-75.051808,3.5,28,71


In [276]:
# text output
for bid in combined_top5['business_id']:
    s_rest = business[business['business_id'] == bid].iloc[0]
    print("{}: {}, with Rating of {}. \n".format(
        s_rest['name'], s_rest['categories'], int(s_rest['stars'])*'⭐'))       

Ahi Sushi: Japanese, Sushi Bars, Restaurants, with Rating of ⭐⭐⭐⭐. 

Tokai Sushi Hibachi Steakhouses: Asian Fusion, Sushi Bars, Japanese, Steakhouses, Restaurants, with Rating of ⭐⭐⭐⭐. 

Hibachi Express: Seafood, Asian Fusion, Chicken Wings, Restaurants, Teppanyaki, Salad, Poke, Japanese, Food, with Rating of ⭐⭐⭐⭐. 

Yamato Japanese Sushi and Hibachi: Steakhouses, Sushi Bars, Japanese, Restaurants, Asian Fusion, with Rating of ⭐⭐⭐⭐. 

New Panda: Restaurants, Japanese, Chinese, with Rating of ⭐⭐⭐. 

